In [1]:
%pylab inline
import time
import numpy as np
from segbench import csaAssign, correspondPixels, matchEdgeMaps2D, matchEdgeMaps3D

Populating the interactive namespace from numpy and matplotlib


In [2]:
data = np.array([[1, 4, 3],
    [2, 5, 3],
    [3, 6, 3],
    [1, 5, 1],
    [2, 6, 1],
    [3, 4, 5]])
n = 6
e1 = csaAssign(n,data)
print e1

[[ 1.  5.  1.]
 [ 2.  6.  1.]
 [ 3.  4.  5.]]


In [3]:
n = 1000
dg = np.random.rand(n,n)>.5
m = np.sum(dg.flat)
i = np.array(dg.ravel().nonzero()).astype(float)
g1 = 1+np.floor(np.divide(i,n))
g2 = (1+np.floor(np.divide(i,n)),
     1+ np.mod(i,n)+n,
     1+np.floor(np.random.rand(1,m))*1000)
g2 = np.array(np.vstack(g2)).T
t = time.time()
e2 = csaAssign(n*2,g2)
print '{:.3e}s'.format(time.time() - t)
esum = np.sum(e2,axis=0)
if not all(esum[0:2]==(n*(n+1)/2,n*(n+1)/2+n*n)): raise ValueError("bug")
if not np.sum(esum[0:2])==(2*n*(2*n+1)/2): raise ValueError("bug")
print n,m, e2[3][:]

8.185e-02s
1000 500225 [  4.00000000e+00   1.00200000e+03   1.00000000e+00]


In [4]:
for n in np.round(np.logspace(0,2,10)).astype(int):
    for thresh in np.linspace(0,1,10):
        print 'n={:d} thresh={:1.2e}'.format(n,thresh),
        bmap1 = (np.random.rand(n,n)>thresh).astype(float)
        bmap2 = (np.random.rand(n,n)>thresh).astype(float)

        match1 = zeros_like(bmap1)
        match2 = zeros_like(bmap2)
        t = time.time()
        cost,oc,match1,match2 = correspondPixels(bmap1,bmap2,.005)
        print '{:1.2e}s'.format(time.time() - t),
        npix = len(bmap1.nonzero()[0]) + len(bmap2.nonzero()[0])
        ncor = len(match1.nonzero()[0]) + len(match1.nonzero()[0])
        nout = len(((bmap1>0)&(match1==0)).nonzero()[0]) + len(((bmap2>0)&(match2==0)).nonzero()[0])
        print npix,ncor,nout
        if ncor+nout!=npix: raise ValueError("bug")
        match1f = match1.ravel()
        idx1 = match1f.astype(int)
        a1 = match1f.nonzero()[0]
        a2 = match1f[idx1.nonzero()[0]].ravel()
        a = vstack((a1,a2)).T


        match2f = match2.ravel()
        idx2 = match2f.astype(int)
        b1 = match2f.nonzero()[0]
        b2 = match2f[idx2.nonzero()[0]].ravel()
        b = vstack((b2,b1)).T
        b = b[b[:,0].argsort(),]
        if sum(a-b): raise ValueError("bug")

n=1 thresh=0.00e+00 1.37e-04s 2 2 0
n=1 thresh=1.11e-01 4.20e-05s 1 0 1
n=1 thresh=2.22e-01 9.99e-05s 2 2 0
n=1 thresh=3.33e-01 5.82e-05s 1 0 1
n=1 thresh=4.44e-01 1.37e-04s 2 2 0
n=1 thresh=5.56e-01 2.50e-05s 1 0 1
n=1 thresh=6.67e-01 4.89e-05s 2 2 0
n=1 thresh=7.78e-01 4.01e-05s 0 0 0
n=1 thresh=8.89e-01 2.00e-05s 1 0 1
n=1 thresh=1.00e+00 4.20e-05s 0 0 0
n=2 thresh=0.00e+00 1.20e-04s 8 8 0
n=2 thresh=1.11e-01 6.70e-05s 6 4 2
n=2 thresh=2.22e-01 7.39e-05s 7 6 1
n=2 thresh=3.33e-01 7.01e-05s 6 4 2
n=2 thresh=4.44e-01 6.48e-05s 5 2 3
n=2 thresh=5.56e-01 7.30e-05s 4 2 2
n=2 thresh=6.67e-01 1.05e-04s 4 2 2
n=2 thresh=7.78e-01 4.39e-05s 2 0 2
n=2 thresh=8.89e-01 7.30e-05s 0 0 0
n=2 thresh=1.00e+00 2.31e-05s 0 0 0
n=3 thresh=0.00e+00 2.70e-04s 18 18 0
n=3 thresh=1.11e-01 2.35e-04s 17 16 1
n=3 thresh=2.22e-01 1.33e-04s 14 10 4
n=3 thresh=3.33e-01 1.18e-04s 13 10 3
n=3 thresh=4.44e-01 1.21e-04s 13 10 3
n=3 thresh=5.56e-01 4.60e-05s 8 4 4
n=3 thresh=6.67e-01 1.81e-05s 2 0 2
n=3 thresh=7.78e-0

In [5]:
for n in np.round(np.logspace(0,2,10)).astype(int):
    for thresh in np.linspace(0,1,2):
        print 'n={:d} thresh={:1.2e}'.format(n,thresh),
        bmap1 = (np.random.rand(n,n)>thresh).astype(float)
        bmap2 = (np.random.rand(n,n)>thresh).astype(float)
        t = time.time()
        cost,match1,match2 = matchEdgeMaps2D(bmap1,bmap2,.005,100)
        print '{:1.2e}s'.format(time.time() - t),
        npix = len(bmap1.nonzero()[0]) + len(bmap2.nonzero()[0])
        ncor = len(match1.nonzero()[0]) + len(match1.nonzero()[0])
        nout = len(((bmap1>0)&(match1==0)).nonzero()[0]) + len(((bmap2>0)&(match2==0)).nonzero()[0])
        print ncor,'+',nout,'=',ncor+nout,'==',npix
        if ncor+nout!=npix: raise ValueError("bug")
        match1f = match1.ravel()
        idx1 = match1f.astype(int)
        a1 = match1f.nonzero()[0]
        a2 = match1f[idx1.nonzero()[0]].ravel()
        a = vstack((a1,a2)).T


        match2f = match2.ravel()
        idx2 = match2f.astype(int)
        b1 = match2f.nonzero()[0]
        b2 = match2f[idx2.nonzero()[0]].ravel()
        b = vstack((b2,b1)).T
        b = b[b[:,0].argsort(),]
        if sum(a-b): raise ValueError("bug")

n=1 thresh=0.00e+00 1.71e-04s 2 + 0 = 2 == 2
n=1 thresh=1.00e+00 6.29e-05s 0 + 0 = 0 == 0
n=2 thresh=0.00e+00 1.36e-04s 8 + 0 = 8 == 8
n=2 thresh=1.00e+00 4.29e-05s 0 + 0 = 0 == 0
n=3 thresh=0.00e+00 2.25e-04s 18 + 0 = 18 == 18
n=3 thresh=1.00e+00 7.10e-05s 0 + 0 = 0 == 0
n=5 thresh=0.00e+00 8.46e-04s 50 + 0 = 50 == 50
n=5 thresh=1.00e+00 7.10e-05s 0 + 0 = 0 == 0
n=8 thresh=0.00e+00 1.53e-03s 128 + 0 = 128 == 128
n=8 thresh=1.00e+00 3.70e-05s 0 + 0 = 0 == 0
n=13 thresh=0.00e+00 5.30e-03s 338 + 0 = 338 == 338
n=13 thresh=1.00e+00 5.32e-05s 0 + 0 = 0 == 0
n=22 thresh=0.00e+00 1.76e-02s 968 + 0 = 968 == 968
n=22 thresh=1.00e+00 7.01e-05s 0 + 0 = 0 == 0
n=36 thresh=0.00e+00 5.81e-02s 2586 + 6 = 2592 == 2592
n=36 thresh=1.00e+00 1.43e-04s 0 + 0 = 0 == 0
n=60 thresh=0.00e+00 1.88e-01s 7196 + 4 = 7200 == 7200
n=60 thresh=1.00e+00 3.22e-04s 0 + 0 = 0 == 0
n=100 thresh=0.00e+00 6.25e-01s 19954 + 46 = 20000 == 20000
n=100 thresh=1.00e+00 8.55e-04s 0 + 0 = 0 == 0


In [8]:
for n in np.round(np.logspace(0,6,4,base=2)).astype(int):
    for thresh in np.linspace(.5,.9,4):
        print 'n={:d} thresh={:1.2e}'.format(n,thresh),
        bmap1 = (np.random.rand(n,n,n)>thresh).astype(float)
        bmap2 = (np.random.rand(n,n,n)>thresh).astype(float)

        t = time.time()
        cost,match1,match2 = matchEdgeMaps3D(bmap1,bmap2,.000005,100)
        print '{:1.4f}s'.format(time.time() - t),
        npix = len(bmap1.nonzero()[0]) + len(bmap2.nonzero()[0])
        ncor = len(match1.nonzero()[0]) + len(match1.nonzero()[0])
        nout = len(((bmap1>0)&(match1==0)).nonzero()[0]) + len(((bmap2>0)&(match2==0)).nonzero()[0])
        print ncor,'+',nout,'=',ncor+nout,'==',npix
        if ncor+nout!=npix: raise ValueError("bug")
        match1f = match1.ravel()
        idx1 = match1f.astype(int)
        a1 = match1f.nonzero()[0]
        a2 = match1f[idx1.nonzero()[0]].ravel()
        a = vstack((a1,a2)).T


        match2f = match2.ravel()
        idx2 = match2f.astype(int)
        b1 = match2f.nonzero()[0]
        b2 = match2f[idx2.nonzero()[0]].ravel()
        b = vstack((b2,b1)).T
        b = b[b[:,0].argsort(),]
        if sum(a-b): raise ValueError("bug")

n=1 thresh=5.00e-01 0.0001s 2 + 0 = 2 == 2
n=1 thresh=6.33e-01 0.0001s 0 + 0 = 0 == 0
n=1 thresh=7.67e-01 0.0001s 0 + 0 = 0 == 0
n=1 thresh=9.00e-01 0.0001s 0 + 0 = 0 == 0
n=4 thresh=5.00e-01 0.0006s 36 + 30 = 66 == 66
n=4 thresh=6.33e-01 0.0003s 20 + 29 = 49 == 49
n=4 thresh=7.67e-01 0.0003s 8 + 21 = 29 == 29
n=4 thresh=9.00e-01 0.0001s 2 + 11 = 13 == 13
n=16 thresh=5.00e-01 0.0501s 2076 + 2053 = 4129 == 4129
n=16 thresh=6.33e-01 0.0223s 1078 + 1911 = 2989 == 2989
n=16 thresh=7.67e-01 0.0074s 418 + 1440 = 1858 == 1858
n=16 thresh=9.00e-01 0.0013s 62 + 760 = 822 == 822
n=64 thresh=5.00e-01 9.7705s 130856 + 131585 = 262441 == 262441
n=64 thresh=6.33e-01 3.9197s 69964 + 121867 = 191831 == 191831
n=64 thresh=7.67e-01 1.2064s 29014 + 93693 = 122707 == 122707
n=64 thresh=9.00e-01 0.1551s 5130 + 47200 = 52330 == 52330
